In [2]:
import pandas as pd
import numpy as np
from datetime import date, time, datetime, timedelta

In [6]:
df2 = pd.read_excel('base1302_pendentes.xlsx')

hoje = date.today()

df2 = df2.drop(['NPC', 'Número do Órgão', 'Sistema Acompanhamento', 'Responsável Cancelamento',
                          'ID Obrigação', 'Observação ', 'Observação da conclusão', 'Projeto', 'Link de Acesso à Audiência'], axis=1)   # dropando as colunas desnecessarias
# criacao de novas colunas vazias para colocar entre as colunas id prazo origem e tem servico contratado
df2[' '] = ' '
df2[' '] = ' '
# selecionar o tipo de atividade = diligencia e nao selecionar os protocolos e apagar todas as linhas
df2_remove = df2[(df2['Tipo de Atividade'] == 'Diligência') &
    (df2['Sub Tipo Compromisso'].str.contains('Protocolo') == False)]

df2 = df2.drop(df2_remove.index)
centro_custo = pd.read_excel('centro_de_custo.xlsx', engine='openpyxl')
df2_ajustado = pd.merge(df2, centro_custo, on='Célula', how='left')
df2_ajustado.drop(['Centro de Custo_x', 'Gestor', 'Diretoria', 'Escritorio'], axis=1, inplace=True)

df2_ajustado = df2_ajustado[['ID Processo', 'ID Prazo', 'Data Cadastro Prazo', 'Data do Prazo','Hora do Prazo', 'Tipo de Atividade', 'Sub Tipo Compromisso','Parte Cliente', 'Parte Adversa', 'Número do Processo', 'Órgão',
       'Comarca', 'UF', 'Tipo de Processo', 'Sistema Eletrônico','Status do Prazo', 'Data Inicio Compromisso', 'Data Conclusão','Data Protocolo', 'Data Auditoria Protocolo', 'Prazo para Protocolo',
       'Data Prazo Automático', 'Data Revisão', 'Prazo Revisão','Data de Reprovação', 'Data Cancelamento','Motivo do cancelamento do Agendamento', 'Observação de Cancelamento',
       'Fase', 'Estratégia', 'Objeto', 'Sub-Objeto', 'Produto','Adv. Responsável Processo', 'Responsável Cadastro','Responsavel Prazo', 'Responsável Revisão', 'Responsável Conclusão',
       'Responsável Protocolo', 'Responsável Auditoria Protocolo', 'Célula','Centro de Custo_y','Cliente', 'Segmento', 'Forma Abertura', 'ID PA', 'ID Prazo Origem',
       ' ', ' ', 'Tem serviço contratado?', 'Id serviço', 'Tipo','Subtipo do serviço', 'Data da contratação', 'Hora da contratação','Status do serviço', 'Status Sugestão alteração/cancelamento',
       'Vinculado ao Recurso?', 'ID Recurso']]

df2_ajustado.rename(columns = {'Centro de Custo_y': 'Centro de Custo'}, inplace=True)

# Acrescentar "Administração Judicial" que não estará mais presente no escritório
df2_final = df2_ajustado[(df2_ajustado['Centro de Custo'] != 'AMBEV') &
            (df2_ajustado['Centro de Custo'] != 'CCB MASSIFICADO') &
            (df2_ajustado['Centro de Custo'] != 'Apresentação QCA') &
            (df2_ajustado['Centro de Custo'] != 'EQUIPE PARAÍBA') &
            (df2_ajustado['Centro de Custo'] != 'MONGERAL') &
            (df2_ajustado['Centro de Custo'] != 'MARITIMO E PORTUARIO') &
            (df2_ajustado['Centro de Custo'] != ' ')]

# Filtragens das datas
filtro1 = (((df2_final['Status do Prazo'] == 'Pendente') | (df2_final['Status do Prazo'] == 'Nao Protocolada')) & (df2_final['Data Prazo Automático'].isnull()))
df2_final.loc[filtro1, "Data Prazo Automático"] = df2_final["Data do Prazo"]

filtro2 = df2_final['Status do Prazo'] == 'Pendente'
df2_final.loc[filtro2, "Data Prazo Automático"] = df2_final["Data do Prazo"]

filtro3 = df2_final['Status do Prazo'] == 'Aguardando Auditoria'
df2_final.loc[filtro3, "Data Prazo Automático"] = df2_final['Prazo para Protocolo'] + timedelta(days = 2)

filtro4 = ((df2_final['Status do Prazo'] == 'Aguardando Revisão') & (df2_final['Prazo Revisão'] < pd.to_datetime(hoje)))
df2_final.loc[filtro4, "Data Prazo Automático"] = df2_final['Prazo para Protocolo']

filtro5 = ((df2_final['Status do Prazo'] == 'Aguardando Revisão') & (df2_final['Prazo Revisão'] >= pd.to_datetime(hoje)))
df2_final.loc[filtro5, "Data Prazo Automático"] = df2_final['Prazo Revisão']

# Fazer a filtragem das datas anteriores a 2020 e remover as linhas
df_remove_data_under_2020 = df2_final['Data Prazo Automático'] < pd.to_datetime(date(2021,1,1))

df2_final = df2_final.drop(df_remove_data_under_2020.index)

# df2_final.to_excel(f'BASE_TRATADA_PENDENTES_{hoje}.xlsx', index=False)

c:\Users\victoramarante\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [113]:
filtro4 = ((df2_final['Status do Prazo'] == 'Aguardando Revisão') & (df2_final['Prazo Revisão'] < pd.to_datetime(hoje)))
df2_final.loc[filtro4, "Data Prazo Automático"] = df2_final['Prazo para Protocolo']

In [5]:
df2_final['Data Prazo Automático'] < pd.to_datetime(date(2021,1,1))

0        False
1        False
2        False
3        False
4        False
         ...  
19677    False
19678    False
19679    False
19680    False
19681    False
Name: Data Prazo Automático, Length: 19610, dtype: bool

In [133]:
pd.to_datetime(datetime.date(2021,12,31))

Timestamp('2021-12-31 00:00:00')